In [1]:
from IPython.display import display, HTML

In [2]:
import pandas as pd
from datetime import datetime

def match_and_format_data(part_a, part_b):
    """
    1、输入数据为 part_a、part_b，结构为：

    part_a={
        "date":["data1","data2","data3"],
        "numbers":[number1,number2,number3]
        }
    part_b={
        "date":["data1","data2","data3"],
        "numbers":[number2,number3，number4]
        }

    2、求两个数据的最佳匹配，匹配规则如下：
    - 输出为表格，有四列（part_a.date, part_a,numbers,part_b,date,part_b.numbers）
    - 求 part_a、part_b 中 numbers 完全匹配的个数最多有多少，限制条件为：
        - part_a.date >= part_b.date
        - part_a.date 和 part_b.date 为同一年
        - 不可交叉匹配，如part_a.date[0] 与 part_b.date[1] 、part_a.date[1] 与 part_b.date[0] 匹配
    - 匹配失败的按下面要求：
        - 未匹配的数据按日期顺序插入到匹配数据前后，保证匹配的数据能在一行就行
        - part_a.date、part_b.date 各自的原始的顺序保持不变
    """
    pass

In [3]:
import pandas as pd
from datetime import datetime

def match_and_format_data(part_a, part_b):
    # Convert dates to datetime and extract years
    a_dates = [pd.to_datetime(d) for d in part_a['date']]
    b_dates = [pd.to_datetime(d) for d in part_b['date']]
    a_years = [d.year for d in a_dates]
    b_years = [d.year for d in b_dates]
    
    m = len(part_a['date'])
    n = len(part_b['date'])
    
    # Initialize DP table
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            a_idx = i-1
            b_idx = j-1
            if (part_a['numbers'][a_idx] == part_b['numbers'][b_idx] and
                a_years[a_idx] == b_years[b_idx] and
                a_dates[a_idx] >= b_dates[b_idx]):
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    
    # Backtrack to find matched pairs
    i, j = m, n
    matched_pairs = []
    while i > 0 and j > 0:
        a_idx = i-1
        b_idx = j-1
        if (part_a['numbers'][a_idx] == part_b['numbers'][b_idx] and
            a_years[a_idx] == b_years[b_idx] and
            a_dates[a_idx] >= b_dates[b_idx] and
            dp[i][j] == dp[i-1][j-1] + 1):
            matched_pairs.append((a_idx, b_idx))
            i -= 1
            j -= 1
        else:
            if dp[i-1][j] > dp[i][j-1]:
                i -= 1
            else:
                j -= 1
    matched_pairs.reverse()
    
    # Sort matched pairs by part_a's date
    sorted_matched_pairs = sorted(matched_pairs, key=lambda x: a_dates[x[0]])
    
    # Prepare result with matched pairs
    result = []
    for pair in sorted_matched_pairs:
        a_idx, b_idx = pair
        a_date = part_a['date'][a_idx]
        a_num = part_a['numbers'][a_idx]
        b_date = part_b['date'][b_idx]
        b_num = part_b['numbers'][b_idx]
        result.append((a_date, a_num, b_date, b_num))
    
    # Collect unmatched a indices
    matched_a_indices = {p[0] for p in matched_pairs}
    unmatched_a = [i for i in range(len(part_a['date'])) if i not in matched_a_indices]
    
    # Insert unmatched a into result
    for i in unmatched_a:
        a_date = part_a['date'][i]
        a_num = part_a['numbers'][i]
        a_dt = a_dates[i]
        inserted = False
        for idx in range(len(result)):
            existing_a_date = result[idx][0]
            if existing_a_date is None:
                continue
            existing_dt = pd.to_datetime(existing_a_date)
            if a_dt < existing_dt:
                result.insert(idx, (a_date, a_num, None, None))
                inserted = True
                break
        if not inserted:
            result.append((a_date, a_num, None, None))
    
    # Collect unmatched b indices
    matched_b_indices = {p[1] for p in matched_pairs}
    unmatched_b = [j for j in range(len(part_b['date'])) if j not in matched_b_indices]
    
    # Insert unmatched b into result
    for j in unmatched_b:
        b_date = part_b['date'][j]
        b_num = part_b['numbers'][j]
        b_dt = b_dates[j]
        inserted = False
        for idx in range(len(result)):
            existing_b_date = result[idx][2]
            if existing_b_date is None:
                continue
            existing_dt = pd.to_datetime(existing_b_date)
            if b_dt < existing_dt:
                result.insert(idx, (None, None, b_date, b_num))
                inserted = True
                break
        if not inserted:
            result.append((None, None, b_date, b_num))
    
    # Create DataFrame
    df = pd.DataFrame(result, columns=['part_a.date', 'part_a.numbers', 'part_b.date', 'part_b.numbers'])
    return df

In [4]:

# 示例数据（与用户提供的相同）
part_a = {
    "date": ["2023-11-21", "2023-11-30", "2023-12-11"],
    "numbers": [250, 150, 100]
}

part_b = {
    "date": ["2023-11-21", "2023-11-30", "2023-11-29"],
    "numbers": [1350, 300, 150]
}

result = match_and_format_data(part_a, part_b)

display(HTML(result.to_html()))

,part_a.date,part_a.numbers,part_b.date,part_b.numbers
0,2023-11-21,250.0,None,NaN
1,None,NaN,2023-11-21,1350.0
2,2023-11-30,150.0,2023-11-29,150.0
3,2023-12-11,100.0,None,NaN
4,None,NaN,2023-11-30,300.0


In [5]:

# 示例数据（与用户提供的相同）
part_a = {
    "date": ["2023-11-21", "2023-11-30", "2023-12-11", "2023-12-25", "2024-01-06", "2024-01-25", "2024-03-06", "2024-03-21", "2024-04-25", "2024-04-30", "2024-05-07", "2024-05-16", "2024-05-28", "2024-06-05", "2024-06-08", "2024-07-12", "2024-07-23", "2024-07-31", "2024-08-12", "2024-08-15", "2024-08-17", "2024-08-20", "2024-08-31", "2024-09-08", "2024-09-10", "2024-09-25", "2024-09-26", "2024-09-30", "2024-10-07", "2024-10-09", "2024-10-11", "2024-10-30", "2024-11-07", "2024-11-14", "2024-11-19", "2024-12-06", "2024-12-14", "2024-12-29", "2024-12-31"],
    "numbers": [250, 150, 100, 90, 390, 156, 80, 174, 70, 100, 100, 65, 115, 100, 160, 70, 50, 55, 35, 30, 40, 50, 25, 65, 100, 45, 55, 20, 20, 100, 80, 70, 50, 60, 150, 70, 76, 50, 76]
}

part_b = {
    "date": ["2023-11-18", "2023-11-28", "2023-11-29", "2023-11-30", "2023-12-5", "2023-12-9", "2023-12-13", "2023-12-23", "2023-12-27", "2024-1-4", "2024-1-5", "2024-1-8", "2024-1-9", "2024-1-10", "2024-1-12", "2024-1-17", "2024-1-20", "2024-3-5", "2024-3-6", "2024-3-8", "2024-3-19", "2024-3-21", "2024-3-22", "2024-4-2", "2024-4-22", "2024-4-23", "2024-4-28", "2024-5-7", "2024-5-18", "2024-5-27", "2024-6-1", "2024-6-5", "2024-6-6", "2024-6-12", "2024-7-6", "2024-7-10", "2024-7-20", "2024-8-1", "2024-8-10", "2024-8-15", "2024-8-16", "2024-8-30", "2024-9-9", "2024-9-11", "2024-9-25", "2024-9-29", "2024-10-6", "2024-10-9", "2024-10-28", "2024-11-6", "2024-11-13", "2024-11-18", "2024-12-5", "2024-12-12", "2024-12-28", "2024-12-29"],
    "numbers": [1350, 300, 150, 200, 150, 100, 100, 90, 100, 200, 994, 200, 100, 100, 100, 240, 306, 280, 140, 136, 150, 24, 65, 214, 145, 70, 310, 295, 65, 115, 90, 100, 180, 160, 20, 70, 50, 55, 35, 40, 80, 25, 65, 100, 100, 20, 120, 80, 70, 50, 60, 150, 70, 76, 50, 76]
}

result = match_and_format_data(part_a, part_b)

display(HTML(result.to_html()))

,part_a.date,part_a.numbers,part_b.date,part_b.numbers
0,2023-11-21,250.0,None,NaN
1,None,NaN,2023-11-18,1350.0
2,None,NaN,2023-11-28,300.0
3,2023-11-30,150.0,2023-11-29,150.0
4,None,NaN,2023-11-30,200.0
5,None,NaN,2023-12-5,150.0
6,2023-12-11,100.0,2023-12-9,100.0
7,None,NaN,2023-12-13,100.0
8,2023-12-25,90.0,2023-12-23,90.0
9,2024-01-06,390.0,None,NaN


In [6]:
part_a={
    "date":["2024-11-13","2024-11-22","2024-12-06","2024-12-07","2024-12-20","2024-12-22","2024-12-29","2024-12-31"],
    "numbers":[797,453,150,250,210,130,60,120]
    }
part_b={
    "date":["2024-10-31","2024-11-20","2024-11-21","2024-12-4","2024-12-6","2024-12-19","2024-12-21","2024-12-28","2024-12-29",],
    "numbers":[800,450,3,150,250,210,130,60,120]
    }

result = match_and_format_data(part_a, part_b)
print(result)

   part_a.date  part_a.numbers part_b.date  part_b.numbers
0   2024-11-13           797.0        None             NaN
1   2024-11-22           453.0        None             NaN
2         None             NaN  2024-10-31           800.0
3         None             NaN  2024-11-20           450.0
4         None             NaN  2024-11-21             3.0
5   2024-12-06           150.0   2024-12-4           150.0
6   2024-12-07           250.0   2024-12-6           250.0
7   2024-12-20           210.0  2024-12-19           210.0
8   2024-12-22           130.0  2024-12-21           130.0
9   2024-12-29            60.0  2024-12-28            60.0
10  2024-12-31           120.0  2024-12-29           120.0


In [7]:
part_a={
    "date":["2024-01-27","2024-11-13","2024-11-29","2024-12-06","2024-12-07","2024-12-21","2024-12-22","2024-12-26"],
    "numbers":[300,650,70,100,100,20,250,119]
    }
part_b={
    "date":["2024-1-23","2024-11-11","2024-11-28","2024-12-4","2024-12-20","2024-12-24","2024-12-29"],
    "numbers":[1600,650,70,200,270,120,1]
    }

result = match_and_format_data(part_a, part_b)
print(result)

   part_a.date  part_a.numbers part_b.date  part_b.numbers
0   2024-01-27           300.0        None             NaN
1         None             NaN   2024-1-23          1600.0
2   2024-11-13           650.0  2024-11-11           650.0
3   2024-11-29            70.0  2024-11-28            70.0
4   2024-12-06           100.0        None             NaN
5   2024-12-07           100.0        None             NaN
6   2024-12-21            20.0        None             NaN
7   2024-12-22           250.0        None             NaN
8   2024-12-26           119.0        None             NaN
9         None             NaN   2024-12-4           200.0
10        None             NaN  2024-12-20           270.0
11        None             NaN  2024-12-24           120.0
12        None             NaN  2024-12-29             1.0


In [8]:
part_a={
    "date":["2024-01-27","2024-11-13","2024-12-01","2024-12-04","2024-12-10","2024-12-26"],
    "numbers":[300,650,70,60,310,90]
    }
part_b={
    "date":["2024-1-23","2024-11-11","2024-11-29","2024-12-9","2024-12-25"],
    "numbers":[1600,650,130,310,90]
    }

result = match_and_format_data(part_a, part_b)
print(result)

  part_a.date  part_a.numbers part_b.date  part_b.numbers
0  2024-01-27           300.0        None             NaN
1        None             NaN   2024-1-23          1600.0
2  2024-11-13           650.0  2024-11-11           650.0
3  2024-12-01            70.0        None             NaN
4  2024-12-04            60.0        None             NaN
5        None             NaN  2024-11-29           130.0
6  2024-12-10           310.0   2024-12-9           310.0
7  2024-12-26            90.0  2024-12-25            90.0


In [9]:
part_a={
    "date":["2024-04-04","2024-04-21","2024-04-22","2024-05-16","2024-06-06","2024-07-12","2024-07-26","2024-08-02","2024-08-03","2024-08-08","2024-08-17","2024-08-20","2024-09-04","2024-09-06","2024-09-21","2024-09-26","2024-10-12","2024-10-20","2024-10-27","2024-10-30","2024-11-08","2024-11-12","2024-11-19"],
    "numbers":[60,15,15,40,50,25,20,8,20,12,10,20,20,20,40,10,9,21,20,20,30,20,30]
    }
part_b={
    "date":["2024-4-2","2024-4-17","2024-4-19","2024-4-23","2024-4-28","2024-5-14","2024-5-16","2024-5-17","2024-5-20","2024-6-6","2024-6-11","2024-7-6","2024-7-10","2024-7-25","2024-8-1","2024-8-2","2024-8-3","2024-8-6","2024-9-21","2024-9-25"],
    "numbers":[280,90,30,20,27,60,20,3,60,120,50,10,25,20,10,8,10,12,40,10]
    }

result = match_and_format_data(part_a, part_b)
print(result)

   part_a.date  part_a.numbers part_b.date  part_b.numbers
0   2024-04-04            60.0        None             NaN
1   2024-04-21            15.0        None             NaN
2   2024-04-22            15.0        None             NaN
3   2024-05-16            40.0        None             NaN
4   2024-06-06            50.0        None             NaN
5   2024-07-12            25.0        None             NaN
6         None             NaN    2024-4-2           280.0
7         None             NaN   2024-4-17            90.0
8         None             NaN   2024-4-19            30.0
9   2024-07-26            20.0   2024-4-23            20.0
10  2024-08-02             8.0        None             NaN
11        None             NaN   2024-4-28            27.0
12        None             NaN   2024-5-14            60.0
13  2024-08-03            20.0   2024-5-16            20.0
14  2024-08-08            12.0        None             NaN
15        None             NaN   2024-5-17             3

In [10]:
display(HTML(result.to_html()))

,part_a.date,part_a.numbers,part_b.date,part_b.numbers
0,2024-04-04,60.0,None,NaN
1,2024-04-21,15.0,None,NaN
2,2024-04-22,15.0,None,NaN
3,2024-05-16,40.0,None,NaN
4,2024-06-06,50.0,None,NaN
5,2024-07-12,25.0,None,NaN
6,None,NaN,2024-4-2,280.0
7,None,NaN,2024-4-17,90.0
8,None,NaN,2024-4-19,30.0
9,2024-07-26,20.0,2024-4-23,20.0


In [11]:
part_a={
    "date":["2023-11-21","2023-11-30","2023-12-11","2023-12-25","2024-01-06","2024-01-25","2024-03-06","2024-03-21","2024-04-25","2024-04-30","2024-05-07","2024-05-16","2024-05-28","2024-06-05","2024-06-08","2024-07-12","2024-07-23","2024-07-31","2024-08-12","2024-08-15","2024-08-17","2024-08-20","2024-08-31","2024-09-08","2024-09-10","2024-09-25","2024-09-26","2024-09-30","2024-10-07","2024-10-09","2024-10-11","2024-10-30","2024-11-07","2024-11-14","2024-11-19","2024-12-06","2024-12-14","2024-12-29","2024-12-31"],
    "numbers":[250,150,100,90,390,156,80,174,70,100,100,65,115,100,160,70,50,55,35,30,40,50,25,65,100,45,55,20,20,100,80,70,50,60,150,70,76,50,76]
    }
part_b={
    "date":["2023-11-18","2023-11-28","2023-11-29","2023-11-30","2023-12-5","2023-12-9","2023-12-13","2023-12-23","2023-12-27","2024-1-4","2024-1-5","2024-1-8","2024-1-9","2024-1-10","2024-1-12","2024-1-17","2024-1-20","2024-3-5","2024-3-6","2024-3-8","2024-3-19","2024-3-21","2024-3-22","2024-4-2","2024-4-22","2024-4-23","2024-4-28","2024-5-7","2024-5-18","2024-5-27","2024-6-1","2024-6-5","2024-6-6","2024-6-12","2024-7-6","2024-7-10","2024-7-20","2024-8-1","2024-8-10","2024-8-15","2024-8-16","2024-8-30","2024-9-9","2024-9-11","2024-9-25","2024-9-29","2024-10-6","2024-10-9","2024-10-28","2024-11-6","2024-11-13","2024-11-18","2024-12-5","2024-12-12","2024-12-28","2024-12-29"],
    "numbers":[1350,300,150,200,150,100,100,90,100,200,994,200,100,100,100,240,306,280,140,136,150,24,65,214,145,70,310,295,65,115,90,100,180,160,20,70,50,55,35,40,80,25,65,100,100,20,120,80,70,50,60,150,70,76,50,76]
    }

result = match_and_format_data(part_a, part_b)
print(result)

   part_a.date  part_a.numbers part_b.date  part_b.numbers
0   2023-11-21           250.0        None             NaN
1         None             NaN  2023-11-18          1350.0
2         None             NaN  2023-11-28           300.0
3   2023-11-30           150.0  2023-11-29           150.0
4         None             NaN  2023-11-30           200.0
..         ...             ...         ...             ...
67  2024-11-19           150.0  2024-11-18           150.0
68  2024-12-06            70.0   2024-12-5            70.0
69  2024-12-14            76.0  2024-12-12            76.0
70  2024-12-29            50.0  2024-12-28            50.0
71  2024-12-31            76.0  2024-12-29            76.0

[72 rows x 4 columns]


In [12]:
display(HTML(result.to_html()))

,part_a.date,part_a.numbers,part_b.date,part_b.numbers
0,2023-11-21,250.0,None,NaN
1,None,NaN,2023-11-18,1350.0
2,None,NaN,2023-11-28,300.0
3,2023-11-30,150.0,2023-11-29,150.0
4,None,NaN,2023-11-30,200.0
5,None,NaN,2023-12-5,150.0
6,2023-12-11,100.0,2023-12-9,100.0
7,None,NaN,2023-12-13,100.0
8,2023-12-25,90.0,2023-12-23,90.0
9,2024-01-06,390.0,None,NaN
